In [3]:
#import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
import random
import sys
import cv2

#데이터 받아오는 경로와 데이터셋 컬럼지정

train_data = pd.read_csv('fififinal.csv')     


train_data = train_data.rename(columns={'class' : 'Class'})


In [5]:
#사용자가 선택한 옷의 info (예를 든 것)
Ex = pd.DataFrame({'성별':[1],
                   '컬러':[7],
                   '톤':[4],
                   '패턴':[10],
                   '상하의':[4],
                   '계절' :[3],
                   'TPO,스타일,무드':['일상,학교,미팅,데이트,파티,남자미니멀,시크'] })
Exx = list(Ex['TPO,스타일,무드'])
Ea = Ex[['성별','계절','상하의']]
train_data = train_data[(train_data['성별'] == int(Ea['성별'])) | (train_data['성별'] == 3) & (train_data['계절'] == int(Ea['계절'])) & (train_data['상하의'] != int(Ea['상하의']))]
TPO = list(train_data['TPO,스타일,무드'])




In [6]:
#tfidf 문서유사도를 이용해 수치화 한 뒤 컬러,톤,패턴 속성과 결합

tfidf_vec = TfidfVectorizer()
X_train_tfidf = tfidf_vec.fit_transform(TPO)
X_test_tfidf = tfidf_vec.transform(Exx)
X_train_tfidf1 = hstack([X_train_tfidf, csr_matrix(train_data[['컬러','톤','패턴']])])
X_test_tfidf1 = hstack([X_test_tfidf, csr_matrix(Ex[['컬러','톤','패턴']])])


In [7]:
#변수 지정
X = X_train_tfidf1
y = np.array(train_data['Class'])
# 학습용, 테스트용 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
#훈련
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
# 클래스 분류값
prediction = clf.predict(X_test_tfidf1)
print(prediction)




[3]


In [8]:
while len(train_data == 0):
    is_venezuela = train_data['Class'] == int(prediction)
    venezuela = train_data[is_venezuela]
    ab = venezuela[['파일명','성별','계절','상하의']]
  
    ac = list(venezuela['파일명'])
    ac = random.choice(ac) 
    updown_name = train_data['파일명'] == ac
    Class_name = train_data['파일명'] == ac
    updown_name = train_data[updown_name]
    Class_name = train_data[Class_name]
    updown_name = updown_name['상하의']
    Class_name = Class_name['Class']
    updown_name = updown_name.values
    Class_name = Class_name.values
    print("상하의 코드 : "+str(updown_name)+",파일명 : "+str(ac))
    image = cv2.imread(str(Class_name[0])+'\\'+str(ac).zfill(6)+'.png')
    im = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    resize_img = cv2.resize(im, (300, 300))
    cv2.imshow('recommend', resize_img)
    cv2.waitKey(0) 
       
    if len(train_data) != 0:
        train_data = train_data[(train_data['상하의'] != int(updown_name))]
    else: 
        sys.exit()

    TPO = list(train_data['TPO,스타일,무드'])
    tfidf_vec = TfidfVectorizer()
    if len(train_data) == 0:
        sys.exit()
    X_train_tfidf = tfidf_vec.fit_transform(TPO)
    X_test_tfidf = tfidf_vec.transform(Exx)
    X_train_tfidf1 = hstack([X_train_tfidf, csr_matrix(train_data[['컬러','톤','패턴']])])
    X_test_tfidf1 = hstack([X_test_tfidf, csr_matrix(Ex[['컬러','톤','패턴']])])
    X = X_train_tfidf1
    y = np.array(train_data['Class'])
            # 학습용, 테스트용 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    prediction = clf.predict(X_test_tfidf1)


















상하의 코드 : [1],파일명 : 2554
상하의 코드 : [4],파일명 : 50
상하의 코드 : [2],파일명 : 6485
상하의 코드 : [5],파일명 : 23740


SystemExit: 

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
